### This notebook is to ingest the relationship between country, ccq, psc, company into cloudant database


In [ ]:
import sys
sys.path.append('/odm_modules')
import pandas as pd
import numpy as np
from common_func import odm_conn

In [ ]:
sql = '''
select  CCOMPAIW, CCOUNTRY||CCOUNTRQ AS CCQ, SUBSTR(RCNUM, 7,3) AS PSC, CFDRSRC, CACTIVE, COUNT(*) as CNT
from ODMPRD.ODMT_EMPLOYEE
WHERE 
1=1
-- AND CFDRSRC IN ('HHA', 'HHP', 'HHE', 'HZA', 'HC9', 'HPH', 'HSW', 'HNW', 'HHI','HHJ')
GROUP BY 
CCOUNTRY||CCOUNTRQ ,CCOMPAIW,SUBSTR(RCNUM, 7,3) , CFDRSRC, CACTIVE
'''
sql_company = """ select CCOMPANY, TCOMPANY FROM ODMPRD.ODMT_ODM_COMPANY WHERE FDISCONT <> 'Y' and CLANGUAG = '' """
sql_ctry = """ select CISOCTRY, TISOCTRY FROM ODMPRD.ODMT_COUNTRYISO WHERE FDISCONT <> 'Y' AND CLANGUAG = '' """
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc(sql)
    result_company = odmprd_adhoc(sql_company)
    result_ctry = odmprd_adhoc(sql_ctry)

In [ ]:
df_comp = pd.DataFrame(result_company).applymap(lambda x: x.strip())
comp_dict = dict(df_comp.values)
df = pd.DataFrame(result)
comp_dict

In [ ]:
ctry_dict = dict(pd.DataFrame(result_ctry).applymap(lambda x: x.strip()).values)
ctry_dict

In [ ]:
# the dataframe for active employee only
#df_a = df.loc[df.CACTIVE == 'A'].sort_values('CCQ').reset_index(drop= True)
df.pivot_table(index = [ 'CCOMPAIW', 'CCQ','PSC', 'CFDRSRC'], 
               columns = 'CACTIVE', 
               values = 'CNT', 
               aggfunc = 'sum', 
               margins = True, 
              margins_name = 'Total').fillna(0).astype(int)

#### relation is to define the relationship between two data fields in odm database.

In [ ]:
def relation(map_from, map_to, df):
    df_temp = df.groupby([map_from, map_to]).CNT.sum().reset_index()
    idx = df_temp.CNT == df_temp.groupby(map_from).CNT.transform(max).reset_index().CNT  # Change == to != , then you can see which record is discarded
    df_temp = df_temp.loc[idx]
    map_dict = dict(df_temp[[map_from, map_to]].values)
    #print(map_dict)
    return map_dict

In [ ]:
df

### get all the stange record based on CCQ

In [ ]:
df_a = df.copy().groupby(['CCOMPAIW', 'CCQ', 'PSC', 'CFDRSRC']).CNT.apply(sum).reset_index()
df_a.to_excel('temp1.xlsx')
threshhold = 10 # set the threshhold = 10
condition = (df_a.CNT != df_a.groupby('CCOMPAIW').CNT.transform(max)) & (df_a.CNT <threshhold)

df_abnormal_comp = df_a.loc[condition].reset_index(drop = True)
df_normal_comp = df_a.loc[~condition].reset_index(drop = True)
df_normal_comp.to_excel('temp3.xlsx')
#df_normal_comp['COMMENT'] = df_normal_comp.CCQ + ' '+ df_normal_comp.PSC + ' '+ df_normal_comp.CFDRSRC + ' '+ df_normal_comp.CNT.astype(str)
df_normal_comp['COMMENT'] = df_normal_comp.apply(lambda row: "Company/{}, CCQ/{}, PSC/{}, Feeder/{}".format(row.CCOMPAIW, row.CCQ, row.PSC, row.CFDRSRC), axis = 1)
df_normal_comp_all = df_normal_comp.copy().drop(columns = ['CNT', 'COMMENT']).drop_duplicates()
df_normal_comp = df_normal_comp.loc[:, ['CCOMPAIW', 'COMMENT']]

df_abnormal_comp.to_excel('temp2.xlsx')


### ----  Put the correct combination of CCQ, COMPANY CODE, PSD AND FEEDER CODE IN on box folder -----

In [ ]:
xlsx_file = 'CCQ_COMP_PSC_COMBINATION.xlsx'
df_normal_comp_all = df_normal_comp_all.reset_index(drop = True).sort_values(by = ['CFDRSRC', 'CCQ'])
df_normal_comp_all['Company Name'] = df_normal_comp_all.CCOMPAIW.apply(lambda x: comp_dict.get(x, ''))
df_normal_comp_all['Country Name'] = df_normal_comp_all.CCQ.apply(lambda x: ctry_dict.get(x[:2], ''))
df_normal_comp_all


https://ibm.ent.box.com/folder/121861696742

In [ ]:
df_normal_comp_all.to_excel(xlsx_file, index = False)

sys.path.append('/app')
from BOX import box_oauth as box
folder_id = '121861696742'
client = box.get_box_client()
box.save2box_folder(client, folder_id, xlsx_file)

In [ ]:
df_normal_comp

In [ ]:
df_merge_comp = df_abnormal_comp.merge(df_normal_comp, left_on = 'CCOMPAIW', right_on = 'CCOMPAIW', how = 'left')
df_merge_comp['COMPANY NAME'] = df_merge_comp.CCOMPAIW.map(comp_dict)
df_merge_comp['COMPANY NAME'] = df_merge_comp['CCOMPAIW'] + ':' + df_merge_comp['COMPANY NAME'] 
df_merge_comp = df_merge_comp.loc[df_merge_comp.CNT < threshhold]
df_merge_comp = df_merge_comp.groupby(['CCOMPAIW', 'CCQ', 'PSC', 'CFDRSRC','CNT', 'COMPANY NAME']).COMMENT.apply(lambda x: "Correct Combination: " + ' OR '.join(x)).reset_index()

In [ ]:
conds = []
for row, value in df_merge_comp.iterrows():
    cond = """ (CCOUNTRY||CCOUNTRQ = '{}' \nAND CCOMPAIW = '{}' \nAND SUBSTR(RCNUM, 7,3) = '{}' \nAND CFDRSRC = '{}'  ) """.format(
        value.CCQ, value.CCOMPAIW, value.PSC, value.CFDRSRC)
    conds.append(cond)

conds = '\n OR \n'.join(conds)
#print(conds)

In [ ]:
 
sql = '''
select 
RCNUM, 
DUPDATE, 
CACTIVE, 
CCOUNTRY||CCOUNTRQ AS CCQ, 
CCOMPAIW, 
CFDRSRC, 
SUBSTR(RCNUM, 7, 3) AS PSC
from ODMPRD.ODMT_EMPLOYEE
WHERE 
{}
--fetch first 100 rows only 
'''.format(conds)
print(sql)
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc(sql)

In [ ]:
df_res = pd.DataFrame(result)
df_final = df_merge_comp.merge(df_res, how = 'outer', on = [ 'CCQ', 'CCOMPAIW', 'CFDRSRC', 'PSC'])
df_final= df_final.loc[:, ['RCNUM', 'DUPDATE', 'CACTIVE', 'CCQ', 'CCOMPAIW', 'PSC', 'CFDRSRC',  'COMMENT',
       'COMPANY NAME' ]].sort_values(by = ['CFDRSRC', 'CCQ', 'DUPDATE']).reset_index(drop = True)
df_final

In [ ]:
xlsx_file_abnormal = 'potential_error_records_in_odm.xlsx'
df_final.to_excel(xlsx_file_abnormal, index = False)


https://ibm.ent.box.com/folder/117883222898

In [ ]:
folder_id = '117883222898'
box.save2box_folder(client, folder_id, xlsx_file_abnormal)

##### get the corresponding feeder code for a given CTRY

In [ ]:
df_a['CTRY'] = df_a.CCQ.str.slice(0,2)
df_temp = df_a.groupby(['CTRY', 'CFDRSRC']).CNT.sum().reset_index()
idx = df_temp.CNT == df_temp.groupby('CTRY').CNT.transform(max).reset_index().CNT  # Change == to != , then you can see which record is discarded
df_temp = df_temp.loc[idx]
dict(df_temp[['CTRY', 'CFDRSRC']].values)

##### get the relationship between company code and feeder code

In [ ]:
df_temp = df_a.groupby(['CCOMPAIW', 'CFDRSRC']).CNT.sum().reset_index()
idx = df_temp.CNT == df_temp.groupby('CCOMPAIW').CNT.transform(max).reset_index().CNT  # Change == to != , then you can see which record is discarded
df_temp = df_temp.loc[idx]
dict(df_temp[['CCOMPAIW', 'CFDRSRC']].values)


In [ ]:
relation('PSC', 'CFDRSRC', df)

In [ ]:
df.loc[(df.CACTIVE == 'N')  & (df.CCQ == 'US '), :]